In [ ]:
import numpy as np
import cv2
import os
import torch
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm.auto import tqdm

In [ ]:
pip install --upgrade ipykernel

In [ ]:
#Grabs images from Roboflow 

from roboflow import Roboflow
rf = Roboflow(api_key="sYsvlRuqf2HHQYxWNXu5")
project = rf.workspace("alzheimer-h49wu").project("alzheimer-detection-ra7oh")
version = project.version(1)
dataset = version.download("folder")